In [1]:
import sys
import numpy as np
import import_ipynb
from scipy.sparse.linalg import spsolve
from pathlib import Path

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))

# Verifique se os caminhos foram adicionados
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_pre_processing import read_mesh
    from fem_processing import boundary_conditions, matrices_assembly
    from fem_pos_processing import graph_results
    print("Modules imports were successful!")
except ModuleNotFoundError as en:
    print(f"Modules were not found: {en}")
except ImportError as en:
    print(f"Error in import: {en}")
    
#run ../setup_project.py

Project root added to sys.path: C:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects
Todas as propriedades foram verificadas com sucesso para os elementos P1!
Todas as propriedades foram verificadas com sucesso para os elementos P2!
Todas as propriedades foram verificadas com sucesso para os elementos P3!
Todas as propriedades foram verificadas com sucesso para os elementos Q1!
Todas as propriedades foram verificadas com sucesso para os elementos Q2!
Modules imports were successful!
Modules imports were successful!
Modules imports were successful!


# Project 1: Poisson Problem at Rectangular Domain $\Omega = [0,1]^2$

Considere um problema de _Poisson_ 2D em $\Omega = [0,1]^2$

$$
-\nabla \cdot \left( \nabla u(x,y) \right) = f(x,y)
\tag{1}
$$

onde,
$$
f(x,y) = 2\pi^2 \, sin(\pi x) \, sin(\pi y)
\tag{2}
$$

cujas condições de contorno são $u=0$ em $\partial \Omega$.

A solução analítica deste problema é

$$
u(x,y) = sin(\pi x) \, sin(\pi y)
\tag{3}
$$

Implemente um programa de elementos finitos utilizando elementos triangulares $P_1$ para resolver o Problema de Valor de Contorno $(1)$.

# 3-noded Linear Triangular Elements, $P_1$
## Pre-processor module
## Geometry and mesh Domain

In [2]:
FINITE_ELEMENT = ("Triangle", 1)
BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'omega'}]
MATERIAL = [{'tag': 201, 'name': 'free_space', 'a_constant': 1}]

# Create mesh from file geometry
ps.create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=1, view_mesh=False)

# Read mesh data
mesh_data = read_mesh.get_data(FINITE_ELEMENT, BOUNDARY, MATERIAL, model='rectangular', info_mode=False)

Model rectangular_domain_Triangle1 (2D)
Info     : 9 geometric entities
Info     : 2 Physical Groups
Info     : 5 nodes in total
Info     : 4 2-D elements in total


## Mesh Data Dictionaries

In [3]:
cell_data = mesh_data['cell']; cell_data

{1: {'Tag': 9,
  'conn': [1, 2, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 2: {'Tag': 10,
  'conn': [4, 1, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 3: {'Tag': 11,
  'conn': [2, 3, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}},
 4: {'Tag': 12,
  'conn': [3, 4, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1}}}

In [4]:
nodes_data = mesh_data['nodes']; nodes_data

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 5: {'xg': (0.5, 0.5), 'bc': {'type': 'Free', 'value': None}}}

In [5]:
conn = {key: cell['conn'] for key, cell in cell_data.items()}; conn

{1: [1, 2, 5], 2: [4, 1, 5], 3: [2, 3, 5], 4: [3, 4, 5]}

## Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$

In [6]:
graph_results.plot_mesh(FINITE_ELEMENT, mesh_data, model='rectangular', Numbering=True)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pre_processing\pictures\rectangular_meshed_domain_Triangle1.svg


<figure>
    <img src="pre_processing/pictures/rectangular_meshed_domain_Triangle1.svg" alt="Fig.1" style="width:100%;" />
    <figcaption>Figure 1: Rectangular meshed domain $\Omega=[0,1]^2$.</figcaption>
</figure>

## `apply_physics()`

In [7]:
mesh_data = ps.apply_physics(FINITE_ELEMENT, mesh_data); mesh_data['cell']

{1: {'Tag': 9,
  'conn': [1, 2, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 9.869604401089356}},
 2: {'Tag': 10,
  'conn': [4, 1, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 9.869604401089356}},
 3: {'Tag': 11,
  'conn': [2, 3, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 9.869604401089356}},
 4: {'Tag': 12,
  'conn': [3, 4, 5],
  'material': {'tag': 201, 'name': 'free_space', 'a_constant': 1},
  'stiffness_a_value': 1,
  'mass_a_value': 1,
  'source': {'type': 'analytical_source_fx', 'value': 9.869604401089356}}}

## ``global_nodes_coordinates()``

In [8]:
nodes_coord = {key: value['xg'] for key, value in nodes_data.items()}; nodes_coord

{1: (0.0, 0.0), 2: (1.0, 0.0), 3: (1.0, 1.0), 4: (0.0, 1.0), 5: (0.5, 0.5)}

## $x_g, y_g$ global coordinate

In [9]:
xg = {key: value['xg'][0] for key, value in nodes_data.items()}; xg

{1: 0.0, 2: 1.0, 3: 1.0, 4: 0.0, 5: 0.5}

## $a_e = (x_e, y_e)$ global element coordinate

In [10]:
ai = {key: [nodes_data[id]['xg'] for id in node_ids] for key, node_ids in conn.items()}
xi = {key: [coord[0] for coord in coords] for key, coords in ai.items()}
yi = {key: [coord[1] for coord in coords] for key, coords in ai.items()}
print("'a_e' global coordinates: "); ai

'a_e' global coordinates: 


{1: [(0.0, 0.0), (1.0, 0.0), (0.5, 0.5)],
 2: [(0.0, 1.0), (0.0, 0.0), (0.5, 0.5)],
 3: [(1.0, 0.0), (1.0, 1.0), (0.5, 0.5)],
 4: [(1.0, 1.0), (0.0, 1.0), (0.5, 0.5)]}

In [11]:
print("'a_1' global coordinates: "); ai[1]

'a_1' global coordinates: 


[(0.0, 0.0), (1.0, 0.0), (0.5, 0.5)]

In [12]:
print("'Node 2' global coordinates: "); nodes_data[2]['xg']

'Node 2' global coordinates: 


(1.0, 0.0)

In [13]:
print("'x_1' x-coordinates: "); xi[1]

'x_1' x-coordinates: 


[0.0, 1.0, 0.5]

## `map_to_physical_coordinates()`

In [14]:
xi_master = (0, 1)
xg_1, yg_1 = matrices_assembly.isomapping_to_global_coordinates(ai[1], xi_master, FINITE_ELEMENT)
print(f"'e_1' Master coordinates: {xi_master} --> Global coordinates: ({xg_1}, {yg_1})")

'e_1' Master coordinates: (0, 1) --> Global coordinates: (0.5, 0.5)


## Material Proprieties, $k_a$

In [15]:
ka = {key: value['stiffness_a_value'] for key, value in cell_data.items()}; ka

{1: 1, 2: 1, 3: 1, 4: 1}

## Source: Analytical source, $f(x)$

In [16]:
fx = {key: value['source']['value'] for key, value in cell_data.items()}; fx

{1: 9.869604401089356,
 2: 9.869604401089356,
 3: 9.869604401089356,
 4: 9.869604401089356}

## Jacobian Matrix Transform

In [17]:
Je = matrices_assembly.jacobian(FINITE_ELEMENT, mesh_data, e=1, xik=(0, 0)) 
Jdet, Jinv = np.linalg.det(Je), np.linalg.inv(Je)
print("Jacobian matrix for element e_1:\n", Je)
print("Determinant of the Jacobian matrix for element e_1:", Jdet)

Jacobian matrix for element e_1:
 [[1.  0. ]
 [0.5 0.5]]
Determinant of the Jacobian matrix for element e_1: 0.5


## Local Elements $e_1$

In [18]:
Ae, fe, Me, area_e = matrices_assembly.local_matrices(FINITE_ELEMENT, mesh_data, e=1)
print("Local stiffness matrix for element e_1:\n", Ae)
print("Local load vector for element e_1:\n", fe)
print("Area of element e_1: ", area_e)

KeyError: 'abc'

## Global “stiffness” matrix, $A_g$

In [19]:
Ag, fg, Mg = matrices_assembly.global_matrix(FINITE_ELEMENT, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (5, 5)


## Imposition of Boundary Conditions
### $Dirichlet$ nodes

In [20]:
dirichlet_nodes = {key: value 
                   for key, value in nodes_data.items() if value['bc']['type'] == 'Dirichlet'}
dirichlet_nodes

{1: {'xg': (0.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 2: {'xg': (1.0, 0.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 3: {'xg': (1.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}},
 4: {'xg': (0.0, 1.0), 'bc': {'type': 'Dirichlet', 'value': 0.0}}}

In [21]:
Nn = len(nodes_data); Nd = len(dirichlet_nodes)
print(f"The entire domain has {Nn} nodes: {Nn - Nd} free nodes; {Nd} Dirichlet nodes.")

The entire domain has 5 nodes: 1 free nodes; 4 Dirichlet nodes.


In [22]:
free_nodes = {key: value for key, value in nodes_data.items() if value['bc']['type'] != 'Dirichlet'}; free_nodes

{5: {'xg': (0.5, 0.5), 'bc': {'type': 'Free', 'value': None}}}

## Mapping Global nodes to reduced system

This code creates a dictionary called ``global_to_reduced``, which maps the global indices of the mesh nodes to the reduced indices, i.e. the indices that correspond only to the nodes that are not in the _Dirichlet boundary conditions_.

In [23]:
{global_id: idx + 1 for idx, global_id in enumerate(free_nodes.keys())}

{5: 1}

## Processor Module
## Asymmetric Global matrix $A_g$ with boundary conditions

In [24]:
Ag, fg = boundary_conditions.apply_simple_dirichlet(Ag, fg, mesh_data)
print("Global matrix shape:", Ag.shape)

Global matrix shape: (5, 5)


## Global Potential Vector

In [25]:
# Make sure Ag is in CSR format for efficiency
# Convert fg to a dense format or keep it sparse as needed
# Solve the linear system Ag * u = fg
ug = spsolve(Ag.tocsr(), fg.toarray())
print("The solution vector u is: \n", ug)

The solution vector u is: 
 [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.82246703+0.j]


## Reduced Global matrix $A_{gr}$ with boundary conditions

In [26]:
Agr, fgr, Mgr = boundary_conditions.reduced_global_matrices(FINITE_ELEMENT, mesh_data)
print("Reduced global matrix shape:", Agr.shape)

Reduced global matrix shape: (1, 1)


## Global Potential Solution

In [27]:
ur = spsolve(Agr.tocsr(), fgr.toarray())
potential_u = boundary_conditions.global_potentials_solution(mesh_data, ur)
print("Reduced solution vector ur: ", ur)
print("Global potential vector V: "); potential_u

Reduced solution vector ur:  [0.82246703+0.j]
Global potential vector V: 


{1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: (0.822467033424113+0j)}

## Solution at each element

In [28]:
u_for_cell = {}
for en, cell in mesh_data['cell'].items():
    u_e = []
    for node in cell['conn']:
        u_e.append(potential_u[node])
    u_for_cell[en] = u_e

print("The complete solution for each cell."); u_for_cell

The complete solution for each cell.


{1: [0.0, 0.0, (0.822467033424113+0j)],
 2: [0.0, 0.0, (0.822467033424113+0j)],
 3: [0.0, 0.0, (0.822467033424113+0j)],
 4: [0.0, 0.0, (0.822467033424113+0j)]}

## Interpolate Solution at element, $e_n$

In [29]:
xi_master = (0, 0.5)
en = 1
u_at_node = matrices_assembly.isomapping_to_global_coordinates(u_for_cell[en], xi_master, FINITE_ELEMENT)

print(f"Element e_{en}: Potential 'u' interpolate at xi = {xi_master} --> u(xi): {u_at_node}")

Element e_1: Potential 'u' interpolate at xi = (0, 0.5) --> u(xi): [0.41123352+0.j]


## Post-Processor

In [ ]:
graph_results.fem_solution(FINITE_ELEMENT, mesh_data, potential_u, type='real')
graph_results.fem_griddata(FINITE_ELEMENT, mesh_data, potential_u, type='real')
ps.plot_analytical_solution(Npts=100)

Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_Triangle1.svg
Arquivo salvo em: c:\Users\adilt\OneDrive\01 ACADEMIA\06 MODELOS\8.FEM\ppgee\projects\p11_rectangular_poisson\pos_processing\pictures\fem_solution_griddata_Triangle1.svg


### Figure 2: Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/fem_solution_Triangle1.svg" alt="Fig.2" style="width:100%;" />
    <figcaption>Figure 2: Post-Processor: Scalar Field Distribution.</figcaption>
</figure>

### Figure 3: Scalar Field Distribution
<figure>
    <img src="pos_processing/pictures/fem_solution_griddata_Triangle1.svg" alt="Fig.3" style="width:100%;" />
    <figcaption>Figure 3: Post-Processor: Scalar Field Distribution and Triangular Mesh.</figcaption>
</figure>

### Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh
<figure>
    <img src="pos_processing/pictures/analytical_solution.svg" alt="Fig.4" style="width:100%;" />
    <figcaption>Figure 4: Analytical Scalar Field Distribution on a Triangular Mesh.</figcaption>
</figure>

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.